In [17]:
import re
import json
from dateparser.search import search_dates

In [2]:
wikipedia_url_regex = re.compile(r'http:\/\/(.*)\.wikipedia\.org\/wiki\/(.*)')

In [3]:
dpr_answer_file = '../data/marriages_wikidata.dpr.answers.pyserini.50k.jl'

In [4]:
def filter_dpr_documents(dpr_answer_file):
    dpr_answers = open(dpr_answer_file)
    filtered_dpr_answers = []
    no_matching_title = []
    for dpr_answer in dpr_answers:
        dpr = json.loads(dpr_answer)
        wiki_title_url = ' '.join(re.search(wikipedia_url_regex, dpr['url']).group(2).lower().split('_'))
        answers = dpr['dpr_answers']
        filtered_answers = []
        for a in answers:
            wiki_title_dpr = a['dpr_answer'].split('\n')[0].lower().replace('"', '')
            if wiki_title_dpr == wiki_title_url:
                filtered_answers.append(a)
        if len(filtered_answers) > 0:
            dpr['filtered_answers'] = filtered_answers
            filtered_dpr_answers.append(dpr)
        else:
            no_matching_title.append(dpr)
    dpr_answers.close()
    print(f'no matching title count: {len(no_matching_title)}')
    return filtered_dpr_answers, no_matching_title

In [5]:
filtered_dpr_answers, no_matching_title = filter_dpr_documents(dpr_answer_file)

no matching title count: 26262


In [6]:
len(filtered_dpr_answers)

23738

In [7]:
def pretty_print_json(json_doc):
    print(json.dumps(json_doc, indent=2))

In [ ]:
pretty_print_json(no_matching_title[436])

In [ ]:
one_para = filtered_dpr_answers[0]['filtered_answers'][0]['dpr_answer']
one_para

In [59]:
def parse_date(para: str) -> str:
    try:
        _dates = search_dates(para,  settings={'PREFER_DAY_OF_MONTH': 'first'})
    except:
        _dates = None
    parsed_dates = []
    if _dates:
        for parsed_date in _dates:
            if parsed_date:
                datetime_obj = parsed_date[1]
                parsed_dates.append((parsed_date[0], f'{datetime_obj.year}-{datetime_obj.month:0>2}-{datetime_obj.day:0>2}'))
    return parsed_dates
    

In [57]:
parse_date(one_para)

[('in 2009', '2009-06-01'),
 ('In August 2013, a', '2013-08-01'),
 ('in 2015', '2015-08-01'),
 ('now', '2015-08-01'),
 ('of February 2016, the', '2016-02-01'),
 ('In 2018', '2018-02-01')]

In [ ]:
c = 1
for f in filtered_dpr_answers:
    if c%1000 == 0:
        print(c)
    c += 1
    dpr_answers = f['dpr_answers']
    for da in dpr_answers:
        parsed_dates = parse_date(da['dpr_answer'])
        if parsed_dates:
            da['parsed_dates'] = parsed_dates
            da['matched'] = False
            for pd in parsed_dates:
                if f['a'] == pd[1]:
                    da['matched'] = True

In [49]:
pretty_print_json(filtered_dpr_answers[0])

{
  "q": "when did Ranbir Kapoor marry Alia?",
  "a": "2022-04-14",
  "n1": "Q1063412",
  "n2": "Q4725343",
  "n1_label": "Ranbir Kapoor",
  "n2_label": "Alia Bhatt",
  "url": "http://en.wikipedia.org/wiki/Ranbir_Kapoor",
  "dpr_answers": [
    {
      "dpr_answer": "\"Ranbir Kapoor\"\nlife. Rumours of an affair with Katrina Kaif first emerged during the production of \"\"Ajab Prem Ki Ghazab Kahani\"\" in 2009. In August 2013, a set of paparazzi photographs of Kapoor and Kaif at a beach in Spain were published by \"\"Stardust\"\". Although Kapoor initially declined to speak of the relationship, he admitted to it in 2015: \"\"Both of us are sure about our relationship and if we don't open about it now, it would be showing disrespect to the relationship.\"\" As of February 2016, the media reported that they had broken up. In 2018, he began dating Alia Bhatt,",
      "dpr_score": 77.88802337646484,
      "dpr_docid": "6329758"
    },
    {
      "dpr_answer": "\"Saif Ali Khan\"\n2007, Kha